In [1]:
import pandas as pd
import numpy as np

In [2]:
from pathlib import Path

In [3]:
dPath = Path("../docs/dumps")

In [4]:
import pickle

In [5]:
with open(dPath / "train_data.pkl", 'rb') as filename:
    train_data = pickle.load(filename)

In [6]:
with open(dPath / "valid_data.pkl", 'rb') as filename:
    valid_data = pickle.load(filename)

In [7]:
X_train = train_data.drop("Detected", axis=1)

In [8]:
y_train = train_data.Detected

In [9]:
X_valid = valid_data.drop("Detected", axis=1)

In [10]:
y_valid = valid_data.Detected

In [11]:
with open(dPath / "rf_exp_02_names.pkl", 'rb') as filename:
    names = pickle.load(filename)

In [12]:
X_train = X_train[names] 

In [13]:
X_valid = X_valid[names]

In [14]:
X_train.head()

,numExecuted,MutatorClass,numAssertInTC,numTestCover,numAssertInTM,ppavcc,cchalsteadCumulativeBugs,cchalsteadCumulativeVolume,ppRVF,ppnumberOfMethods,...,cchalsteadEffort,ccmaintainabilityIndexNC,mmhalsteadVocabulary,pptcc,mmexternalMethodsCalled,ppinstability,ppmaxcc,mmhalsteadEffort,mmvariablesReferenced,ccunweightedClassSize
0,14,2,14,1,7,2.125,0.593518,1780.554266,13.0,16.0,...,32644.005473,99.687474,21.0,34.0,1.0,0.666667,5.0,2782.972319,3.0,4.0
1,14,6,14,1,7,2.125,0.593518,1780.554266,13.0,16.0,...,32644.005473,99.687474,21.0,34.0,1.0,0.666667,5.0,2782.972319,3.0,4.0
2,14,6,14,1,7,2.125,0.593518,1780.554266,13.0,16.0,...,32644.005473,99.687474,21.0,34.0,1.0,0.666667,5.0,2782.972319,3.0,4.0
3,14,0,14,1,7,2.125,0.593518,1780.554266,13.0,16.0,...,32644.005473,99.687474,21.0,34.0,1.0,0.666667,5.0,2782.972319,3.0,4.0
4,14,0,14,1,7,2.125,0.593518,1780.554266,13.0,16.0,...,32644.005473,99.687474,21.0,34.0,1.0,0.666667,5.0,2782.972319,3.0,4.0


In [15]:
from imblearn.over_sampling import SMOTE

In [16]:
sm = SMOTE(random_state=42, n_jobs=-1, k_neighbors=3)

In [17]:
%time X_res, y_res = sm.fit_resample(X_train, y_train)

Wall time: 16.2 s


In [18]:
from sklearn.ensemble import RandomForestClassifier

In [19]:
rf = RandomForestClassifier(n_estimators=20, max_features=0.7, oob_score=True, random_state=42, n_jobs=-1)

In [20]:
%time rf.fit(X_res, y_res)

Wall time: 2min 2s


C:\Users\Alireza\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:523: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
C:\Users\Alireza\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:528: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features=0.7,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=-1,
                       oob_score=True, random_state=42, verbose=0,
                       warm_start=False)

In [21]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report

In [22]:
def print_score(m):
    res = [m.score(X_train, y_train), m.score(X_valid, y_valid), roc_auc_score(y_valid, m.predict(X_valid))]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [23]:
def conf_matr(m):
    y_pred = m.predict(X_valid)
    print(classification_report(y_valid, y_pred))

In [24]:
print_score(rf)

[0.916949782669741, 0.6954041720990873, 0.5819920275331714, 0.868249925787025]


In [25]:
conf_matr(rf)

              precision    recall  f1-score   support

       False       0.38      0.36      0.37     16657
        True       0.79      0.81      0.80     50839

    accuracy                           0.70     67496
   macro avg       0.58      0.58      0.58     67496
weighted avg       0.69      0.70      0.69     67496



In [26]:
with open(dPath / "rf_exp_04.pkl", 'wb') as filename:
    pickle.dump(rf,filename)

In [27]:
names = X_train.columns

In [28]:
feat_importances = pd.Series(rf.feature_importances_, index=names)
feat_importances.sort_values(ascending=False,inplace=True)

In [29]:
feat_importances

numExecuted                      0.124785
MutatorClass                     0.103495
numAssertInTC                    0.069502
numAssertInTM                    0.048317
ppavcc                           0.045408
numTestCover                     0.043002
ppnumberOfMethods                0.030521
cchalsteadCumulativeBugs         0.028791
mmhalsteadDifficulty             0.026402
mmnumberOfExpressions            0.021883
mmhalsteadVocabulary             0.021425
mmhalsteadEffort                 0.020964
ppmaxcc                          0.020465
ppmaintainabilityIndexNC         0.019560
mminstanceVariablesReferenced    0.018901
cchalsteadCumulativeVolume       0.018583
ppnumberOfClasses                0.018434
mmvariablesReferenced            0.018191
ppmaintainabilityIndex           0.018056
ccmaintainabilityIndex           0.017853
mmexternalMethodsCalled          0.017683
ccmaintainabilityIndexNC         0.017435
ppRVF                            0.016789
ccunweightedClassSize            0

In [30]:
with open(dPath / "rf_exp_04_names.pkl", 'wb') as filename:
    pickle.dump(names,filename)